# Abrindo espectros 1D

In [1]:
import nmrglue as ng ## Python NMR Processing package 
import matplotlib.pyplot as plt ## used to plot graphics in pyhon
#%matplotlib qt5

##NA0048214682_3620_20221122_01/PROTON_01.fid

name="NA0048214946_3492_20220906_01/PROTON_01.fid"
# read in the Varian/Agilent formatted data
dic, data = ng.varian.read(name)


np=dic['np']  ## GET Number of acquired points in Varian Spectra
lb=0.3 # Hz  ## For exponencial multiplication 
swh=float(dic['procpar']['sw']['values'][0]) ## Sweep Width in Hz 

data = ng.proc_base.zf_size(data, np*2) # zero fill to 2x NP points
data = ng.process.proc_base.em(data, lb=lb/swh, inv=False, rev=False) ## apodization
data = ng.proc_base.fft(data)               # Fourier transform

data = ng.proc_autophase.autops(data, 'peak_minima', p0=-115., p1=44.)
data = ng.process.proc_autophase.autops(data, fn='acme', p0=0, p1=0, return_phases=False, disp=False) # auto phase correction
data = ng.proc_base.di(data)                # discard the imaginaries to plot

si=len(data) ## number of processed points

sf=float(dic['procpar']['H1reffrq']['values'][0])  ## Spectrometer frequency      
sfo1=float(dic['procpar']['sfrq']['values'][0])  ## Central frequency in MHz
o1=(sfo1-sf)*1e6 ## Central Frequency in Hz
hzppt=float(dic['procpar']['sw']['values'][0])/si ## Hertz per processed point
sr=o1+(sf-sfo1)*1000000. ## Spectrometer reference peak in Hz
pts=int(sr//hzppt) # Calc pts to Calibrate 0ppm to Xscale

data=data/data.max()*100

xs=[] ## array used to correct X scale

### Block to rescale X
import numpy as np
for j in range(0-pts,si-pts):
    hz=float(((o1-swh/2)+(hzppt*(j)))/sf)
    xs+=[hz]
### End Block to rescale X 

xs = np.asarray(xs) ## transform xs in numpy array
ppm_of_reference_peak=1.86813 ### Needs to correct by hands on all new spectra
xarr=xs.max()-xs-ppm_of_reference_peak #### peaks calibration according to TMS or Solvent position
treshold=10
peak_table = ng.peakpick.pick(data, pthres=data.max()/treshold, algorithm='downward') ## peak table according to treshold 
udic = ng.varian.guess_udic(dic, data) ### Create a Universal dictionary from Varian procpar
# create a unit conversion object
uc = ng.fileiobase.uc_from_udic(udic)


# convert the peak locations from unit of points to PPM
# using the unit conversion object
peak_locations_ppm = [xarr[int(i)] for i in peak_table['X_AXIS']]

# Find the peak amplitudes
peak_amplitudes = data[peak_table['X_AXIS'].astype('int')]

## Use this block to test in Jupyter Lab
#fig = plt.figure()
#ax = fig.add_subplot(111)
#ax.invert_xaxis()
#ax.plot(xarr, data)

#ax.plot(peak_locations_ppm, peak_amplitudes, 'ro')



Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 50
         Function evaluations: 101


In [2]:
decimals=4 # number of decimal places for printed peaks

Peaks=np.around(peak_locations_ppm,decimals) ## we can use this as a start point of assignments
print(len(Peaks))

73


In [13]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show,  output_file
from bokeh.models import Legend,  Label,  Text,HoverTool, Arrow,  VeeHead,  OpenHead, Div, Row
from bokeh.core.properties import value
from bokeh.models import BoxSelectTool

output_file('index.html') ## used as final HTML file 

title="Embrapa Copaiba Database Collection"


pwidth=1024 ## HTML page width size/resolution
pheight=768 ## HTML page height size/resolution

## Block used to calc X and Y scale 
xmin = xarr.min()
xmax = xarr.max()
## plot y min as 0.1%
ymin = data.min()/1000
## Plot Y as 20% of max
ymax = data.max()/1

HoverTIPS = [
    ("(Peak in ppm)", "($x)"),
    ("Intensity", " $y"),
]

##  Setup page range
p = figure(x_range=(xmax, xmin), 
           y_range=(ymin, ymax),
           width=pwidth, 
           height=pheight,
           tools="pan, box_select,box_zoom,wheel_zoom,undo,redo,reset,save,hover",
           tooltips=HoverTIPS,
           #y_axis_type="log",
           title= title)



## plot peaks at spectra
for i in range (0,len(peak_locations_ppm)):
    if Peaks[i]>2.755 and Peaks[i]<3.05:
        p.add_layout(Arrow(end=OpenHead(line_color="red" ,size=10),
                           line_color="red",
                           line_width=2,
                           x_start=peak_locations_ppm[i], 
                           y_start=peak_amplitudes[i]*1.55, 
                           x_end=peak_locations_ppm[i], 
                           y_end=peak_amplitudes[i]*1.21))
        citation=Label(x=peak_locations_ppm[i], 
                       y=peak_amplitudes[i]*1.57, 
                       text="Citrate, "+str(Peaks[i]), 
                       text_font_size="10pt",
                       text_font_style = "italic",
                       text_align="center",
                       text_color='red',
                       angle_units = 'deg',
                       angle=90)
        p.add_layout(citation)
    else:
        peaking = Label(
            x=peak_locations_ppm[i], 
            y=peak_amplitudes[i],
            text=str(Peaks[i]), 
            #render_mode='css',
            border_line_color='blue', 
            border_line_alpha=0.0,
            angle_units = 'deg',
            angle=90,
            background_fill_color='white', 
            background_fill_alpha=0.0,
            text_font="Arial",
            text_font_size='8pt',
                   )
        p.add_layout(peaking)
    

## plot spectra
p.line(
    xarr,
    data,
    line_width=2, 
    color='blue',
    legend_label=name
    )

p.scatter(
    peak_locations_ppm, 
    peak_amplitudes, 
    color='red',
    size=8
    )


## Background Setup Block
#p.background_fill_color= "blue"
#p.background_fill_alpha = 0.15
#url="layout_set_logo.png"
#p.image_url(url=value(url), x=xmax/2, y=ymax/2, alpha=0.7, anchor="center",
#            w=1024/2, w_units="screen", h=768/2, h_units="screen")

## Setup title
p.title.text_font = 'Courier'
p.title.text_font_size = '20pt'

## Setup legend 
p.legend.location = "top_left"
p.legend.label_text_font = "Arial"
p.legend.label_text_font_style = "italic"
p.legend.label_text_font_size = "10pt"
p.legend.orientation = "horizontal"
p.legend.click_policy="mute"


## Setup Xaxis fonts
## Setup Xaxis fonts
p.xaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font="Arial"
p.xaxis.major_label_text_font_size="14pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.axis_label = "$$^1H\ ppm$$"

## Setup yaxis fonts
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_label = "$$Intensity\ scaled(\%)$$"

#show(p) # show the results
url = "https://www.embrapa.br/documents/1355291/47137384/Logomarca+Embrapa.png/2999900a-8c32-96f5-8f73-1a591bba1ee1?t=1570557387741"
d1 = Div(text = '<div style="position: absolute; left:-900px; top:80px"><img src=' + 
         url + ' style="width:241px; height:171px; opacity: 0.5"></div>')

show(Row(p, d1))



